# McNemar Test

## Import All Necessary Libraries

In [39]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar 
import os

## Read All the Data

In [40]:
dir = '/Users/angelaoryza/Documents/TA/noisy-rnnids/rnnids-py/results/vector/mcnemar/ftp'
subdir = os.listdir(dir)

In [41]:
df_dict = {}
for file in subdir:
    if file.endswith('.csv'):
        df = pd.read_csv(f'./{file}')
        file = file.split('.csv')[0]
        df_dict[file] = df

In [42]:
df_dict['lstm_pred_0.0']

,Unnamed: 0,id,b_mean,b_iqr,b_zscore,f_zscore,f_mean,f_iqr
0,0,175.45.176.0-45235-149.171.126.16-21-tcp,1,1,1,1,1,1
1,1,175.45.176.3-1025-149.171.126.11-21-tcp,1,1,1,1,1,1
2,2,175.45.176.1-28136-149.171.126.11-21-tcp,1,1,1,1,1,1
3,3,175.45.176.2-43663-149.171.126.15-21-tcp,1,1,1,1,1,1
4,4,175.45.176.3-64709-149.171.126.15-21-tcp,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
25298,25298,59.166.0.8-55795-149.171.126.6-21-tcp,0,0,0,0,0,0
25299,25299,149.171.126.1-21-59.166.0.7-7753-tcp,0,1,1,1,0,1
25300,25300,59.166.0.3-7585-149.171.126.2-21-tcp,0,0,0,0,0,0
25301,25301,59.166.0.1-27560-149.171.126.3-21-tcp,0,1,1,1,0,1


In [43]:
for key, df in df_dict.items():
    df.drop('Unnamed: 0', axis=1, inplace=True)

## Data Exploration

In [44]:
for key, df in df_dict.items():
    temp = df[df.duplicated()]['id'].unique()
    print(f'There are {len(temp)} duplicates data in df {key}')

There are 388 duplicates data in df lstm_pred_0.002
There are 378 duplicates data in df lstm_pred_0.003
There are 390 duplicates data in df bi-lstm_pred_0.0039
There are 386 duplicates data in df bi-lstm_pred_0.0005
There are 387 duplicates data in df lstm_pred_0.001
There are 505 duplicates data in df lstm_pred_0.0
There are 385 duplicates data in df bi-lstm_pred_0.0015
There are 389 duplicates data in df lstm_pred_0.0035
There are 389 duplicates data in df lstm_pred_0.0025
There are 386 duplicates data in df lstm_pred_0.0015
There are 505 duplicates data in df bi-lstm_pred_0.0
There are 389 duplicates data in df lstm_pred_0.0039
There are 386 duplicates data in df lstm_pred_0.0005
There are 392 duplicates data in df bi-lstm_pred_0.0025
There are 376 duplicates data in df bi-lstm_pred_0.003
There are 389 duplicates data in df bi-lstm_pred_0.002
There are 493 duplicates data in df pred_0.0
There are 390 duplicates data in df bi-lstm_pred_0.0035
There are 386 duplicates data in df bi-ls

In [45]:
for key, df in df_dict.items():
    count = 0
    for t in temp:
        srs = df[df['id']==t][['b_mean', 'b_iqr', 'b_zscore', 'f_mean', 'f_iqr', 'f_zscore' ]].nunique()
        if (srs > 1).any():
            print(t)
            count += 1
            break
        else:
            continue
    if count == 0:
        print(f'All duplicated values in df {key} have the same prediction')
    else:
        print(f'Not all duplicated values in df {key} have the same prediction')

175.45.176.2-1033-149.171.126.16-21-tcp
Not all duplicated values in df lstm_pred_0.002 have the same prediction
175.45.176.2-1033-149.171.126.16-21-tcp
Not all duplicated values in df lstm_pred_0.003 have the same prediction
175.45.176.1-52899-149.171.126.10-21-tcp
Not all duplicated values in df bi-lstm_pred_0.0039 have the same prediction
175.45.176.0-2939-149.171.126.15-21-tcp
Not all duplicated values in df bi-lstm_pred_0.0005 have the same prediction
149.171.126.6-21-59.166.0.7-61162-tcp
Not all duplicated values in df lstm_pred_0.001 have the same prediction
149.171.126.0-21-59.166.0.9-40745-tcp
Not all duplicated values in df lstm_pred_0.0 have the same prediction
175.45.176.2-1033-149.171.126.16-21-tcp
Not all duplicated values in df bi-lstm_pred_0.0015 have the same prediction
175.45.176.1-1238-149.171.126.13-21-tcp
Not all duplicated values in df lstm_pred_0.0035 have the same prediction
175.45.176.2-1033-149.171.126.16-21-tcp
Not all duplicated values in df lstm_pred_0.0025

## Data Pre-Processing

In [46]:
def preprocess(df):
    df = df.sort_values(by='id')
    df.drop_duplicates(inplace=True)
    return df

In [47]:
for key, df in df_dict.items():
    df = preprocess(df)

### Merge DF

In [48]:
df_merge = dict()
x = df_dict['lstm_pred_0.0']
for key, df in df_dict.items():
    if key == 'lstm_pred_0.0':
        continue
    else:
        df_all = pd.merge(x, df, on='id', how='inner')
        df_all = df_all.sort_values(by='id')
        df_merge[key] = df_all

- Plot F1-Score untuk tiap tiap threshold
- Cari trend regresinya, Hitung gradiennya untuk tiap threshold.
- Dilakukan untuk setiap protokol
- Rekap McNemar

- Vectorizing:


## Run McNemar Test

In [49]:
def mcnemar_test(df, x , y, key):
    a = len(df[(df[x]==0) & (df[y]==0)])
    b = len(df[(df[x]==0) & (df[y]==1)])
    c = len(df[(df[x]==1) & (df[y]==0)])
    d = len(df[(df[x]==1) & (df[y]==1)])

    table = np.array([
    [a,b],
    [c,d]
    ])

    print(key)
    print(table)
    result = mcnemar(table)
    print(result)
    return result.pvalue


### For Binary Score

#### For B-Mean

In [50]:
result = {}
result['b-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_mean_x', 'b_mean_y', key)
    result['b-mean'][key] = test

lstm_pred_0.002
[[23762    31]
 [  847  2107]]
pvalue      1.2984837671365309e-207
statistic   31.0
lstm_pred_0.003
[[23441   352]
 [  779  2175]]
pvalue      1.2010411907252253e-37
statistic   352.0
bi-lstm_pred_0.0039
[[23768    25]
 [ 1119  1835]]
pvalue      0.0
statistic   25.0
bi-lstm_pred_0.0005
[[23747    46]
 [  309  2645]]
pvalue      5.632666389647594e-49
statistic   46.0
lstm_pred_0.001
[[23753    40]
 [  794  2160]]
pvalue      6.11864796857898e-183
statistic   40.0
bi-lstm_pred_0.0015
[[23757    36]
 [  792  2162]]
pvalue      1.6279146300642888e-186
statistic   36.0
lstm_pred_0.0035
[[23764    29]
 [ 1002  1952]]
pvalue      1.6488282515213585e-254
statistic   29.0
lstm_pred_0.0025
[[23758    35]
 [  796  2158]]
pvalue      1.0496176629224177e-188
statistic   35.0
lstm_pred_0.0015
[[23751    42]
 [  821  2133]]
pvalue      1.8166007866960226e-188
statistic   42.0
bi-lstm_pred_0.0
[[23751    42]
 [  539  2415]]
pvalue      5.3200910567707045e-111
statistic   42.0
lstm_pre

#### For B-IQR

In [51]:
result['b-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_iqr_x', 'b_iqr_y', key)
    result['b-iqr'][key] = test

lstm_pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0025
[[21298     3]

#### For B-ZScore

In [52]:
result['b-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_zscore_x', 'b_zscore_y', key)
    result['b-zscore'][key] = test

lstm_pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0025
[[21298     3]

### For Floating Score

#### For F-Mean

In [53]:
result['f-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_mean_x', 'f_mean_y', key)
    result['f-mean'][key] = test

lstm_pred_0.002
[[24849   104]
 [  312  1482]]
pvalue      3.1348634963302636e-25
statistic   104.0
lstm_pred_0.003
[[24384   569]
 [  490  1304]]
pvalue      0.01649484012099444
statistic   490.0
bi-lstm_pred_0.0039
[[24784   169]
 [  375  1419]]
pvalue      5.705045305007516e-19
statistic   169.0
bi-lstm_pred_0.0005
[[24948     5]
 [  281  1513]]
pvalue      2.5210161089012967e-76
statistic   5.0
lstm_pred_0.001
[[24924    29]
 [  297  1497]]
pvalue      3.880967585341674e-57
statistic   29.0
bi-lstm_pred_0.0015
[[24895    58]
 [  285  1509]]
pvalue      3.987038120055932e-37
statistic   58.0
lstm_pred_0.0035
[[24883    70]
 [  357  1437]]
pvalue      2.0196426581239394e-47
statistic   70.0
lstm_pred_0.0025
[[24833   120]
 [  311  1483]]
pvalue      1.2804883440153633e-20
statistic   120.0
lstm_pred_0.0015
[[24901    52]
 [  292  1502]]
pvalue      1.1508829915370687e-41
statistic   52.0
bi-lstm_pred_0.0
[[24929    24]
 [   13  1781]]
pvalue      0.09887174959294498
statistic   13.0


#### For F-IQR

In [54]:
result['f-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_iqr_x', 'f_iqr_y', key)
    result['f-iqr'][key] = test

lstm_pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0025
[[21298     3]

#### For F-ZScore

In [55]:
result['f-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_zscore_x', 'f_zscore_y', key)
    result['f-zscore'][key] = test

lstm_pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
lstm_pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
bi-lstm_pred_0.0025
[[21298     3]

## Saving Test Result

In [56]:
mcnemar_result = pd.DataFrame.from_dict(result).sort_index(axis=0)
mcnemar_result

,b-mean,b-iqr,b-zscore,f-mean,f-iqr,f-zscore
bi-lstm_pred_0.0,5.320091e-111,1.0,1.0,9.887175e-02,1.0,1.0
bi-lstm_pred_0.0005,5.632666e-49,1.0,1.0,2.521016e-76,1.0,1.0
bi-lstm_pred_0.001,3.335184e-198,1.0,1.0,5.489163e-60,1.0,1.0
bi-lstm_pred_0.0015,1.627915e-186,1.0,1.0,3.987038e-37,1.0,1.0
bi-lstm_pred_0.002,6.537331e-212,1.0,1.0,1.060469e-24,1.0,1.0
bi-lstm_pred_0.0025,3.094975e-244,1.0,1.0,8.847677e-21,1.0,1.0
bi-lstm_pred_0.003,1.418591e-51,1.0,1.0,2.436338e-17,1.0,1.0
bi-lstm_pred_0.0035,6.222313e-267,1.0,1.0,2.150339e-44,1.0,1.0
bi-lstm_pred_0.0039,0.000000e+00,1.0,1.0,5.705045e-19,1.0,1.0
lstm_pred_0.0005,8.878902e-16,1.0,1.0,2.159339e-73,1.0,1.0


In [57]:
mcnemar_result.to_excel('./ftp-mcnemar-lstm.xlsx')